In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
#pulling data API
key =credentials.login['key_isaham']  # extract password

In [2]:
# Define the instruments to download.
#tech = ["GREATEC", 
       #"INARI"]

tech = ["GREATEC",
        "UWC",       
        "D&O", 
       "GENETEC", 
       "UNISEM", 
       "FRONTKN", 
       "ATAIMS", 
       "FPI", 
       "PENTA", 
       "DUFU", 
       "VS", 
       "SKPRES", 
        "MPI", 
        "VITROX"
       ] 

       
# looping through all stocks
# dictionary of dataframes
ss={}
d = {}
# listing all stocks
all_stocks = [] 
mk={}


for m,stock in enumerate (tech):
        
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 
    
    df['Stock_Name'] = symbol

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    #print(oversold[oversold == True])
    #print(df)
    mk["{}".format(stock)] = df
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    vol = []
    ope =[]
    high = []
    low = []


    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        vol.append(df['v'].loc[i])
        ope.append(df['o'].loc[i])
        high.append(df['h'].loc[i])
        low.append(df['l'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        del(ope[-diff:])
        del(high[-diff:])
        del(vol[-diff:])
        del(low[-diff:])
        #print("Position closed")
    dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Open":ope,
        'Volume': vol,
        "Low": low,
        "high": high,
        "Pct_change": pct    
    }  
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    #Signal_buy = []
    #for t in range(len(trans)):
        #Signal_buy.append(1)
        
    
    trans['Stock'] = pd.Series(Name_stock)
    #trans['Signal'] = pd.Series(Signal_buy)
    
    # scaling certain features only
    col_names = ['Buy', 'Low', 'Open', 'Volume','high']  
    features = trans[col_names]
    
    #features = dataf[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    # convert the array back to a dataframe
    dataset = DataFrame(features)
    
    
    col_notscale=['Buy_Date','Stock','wr']
    features_notscale=trans[col_notscale]
    
    # saving in multiple df
    d["{}".format(stock)] = dataset
    ss["{}".format(stock)] = features_notscale
    
    # list of all stocks
    all_stocks.append(d[stock])
    
    
    #print(trans)
    
    #dct_1={ 'Number of stock':dct
    
    #}
    #trans_1 = pd.DataFrame(dct_1)
    
    #print(trans[0:])
    
    #trans.to_csv('result {}.csv'.format(symbol), index=False)
    #C:\Users\Mo Syamil\Documents\data scientiest trial\strategy stho\result
            
    print("----------------DONE {}------------------".format(symbol))
    #print(trans)
    #concat_all= pd.concat([trans], axis=0)
    #print(concat_all)
    #data_1=[]
    
    #for d in  range(len(trans)):
        #concat_all=pd.concat([trans], axis=0)
        #data_1.append(list(trans))
        #print(data_1)
    #c_data= trans[m]
    #print(concat_all)
    #print(trans)
    
    
        
    #print(trans)
    #print()

    
# Merging all df
#concat_all= pd.concat([trans], axis=0)
#print(trans.append(m))
#print(trans)
#print(concat_all)

#print("Finish")

    



----------------PROCESSING GREATEC------------------
----------------DONE GREATEC------------------
----------------PROCESSING UWC------------------
----------------DONE UWC------------------
----------------PROCESSING D&O------------------
----------------DONE D&O------------------
----------------PROCESSING GENETEC------------------
----------------DONE GENETEC------------------
----------------PROCESSING UNISEM------------------
----------------DONE UNISEM------------------
----------------PROCESSING FRONTKN------------------
----------------DONE FRONTKN------------------
----------------PROCESSING ATAIMS------------------
----------------DONE ATAIMS------------------
----------------PROCESSING FPI------------------
----------------DONE FPI------------------
----------------PROCESSING PENTA------------------
----------------DONE PENTA------------------
----------------PROCESSING DUFU------------------
----------------DONE DUFU------------------
----------------PROCESSING VS---------

In [3]:
# combining all data generated
df_notscale = pd.concat(list(ss.values()),ignore_index=True)

In [4]:
df_notscale

,Buy_Date,Stock,wr
0,2019-11-25,GREATEC,1
1,2019-11-27,GREATEC,1
2,2020-03-10,GREATEC,0
3,2020-09-11,GREATEC,1
4,2020-09-18,GREATEC,1
...,...,...,...
1340,2020-05-21,VITROX,1
1341,2020-09-11,VITROX,1
1342,2020-11-27,VITROX,1
1343,2021-02-24,VITROX,0


In [5]:
df_scale = pd.concat(list(d.values()),ignore_index=True)

In [6]:
df_scale=df_scale.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})

In [7]:
df_scale

,Buy,Low,Open,Volume,High
0,-1.422847,-1.398726,-1.394648,-0.218389,-1.426193
1,-1.437665,-1.416967,-1.397626,0.867358,-1.423269
2,-1.102767,-1.176798,-1.177276,2.146298,-1.098748
3,-0.213656,-0.261726,-0.239299,0.782518,-0.215816
4,-0.169201,-0.155322,-0.179745,-1.236101,-0.192427
...,...,...,...,...,...
1340,1.418559,1.449847,1.438214,-0.257703,1.449488
1341,2.533699,2.438409,2.432330,0.540154,2.500931
1342,3.224771,3.202541,3.181903,-0.426413,3.262321
1343,3.826842,3.827740,3.846419,-0.301918,3.780274


In [8]:
#df_scale=df_scale[['Buy', 'Low', 'Open', 'Volume','High']]

In [9]:
result_scaler = pd.concat([df_notscale, df_scale], axis=1)

In [10]:
result_scaler

,Buy_Date,Stock,wr,Buy,Low,Open,Volume,High
0,2019-11-25,GREATEC,1,-1.422847,-1.398726,-1.394648,-0.218389,-1.426193
1,2019-11-27,GREATEC,1,-1.437665,-1.416967,-1.397626,0.867358,-1.423269
2,2020-03-10,GREATEC,0,-1.102767,-1.176798,-1.177276,2.146298,-1.098748
3,2020-09-11,GREATEC,1,-0.213656,-0.261726,-0.239299,0.782518,-0.215816
4,2020-09-18,GREATEC,1,-0.169201,-0.155322,-0.179745,-1.236101,-0.192427
...,...,...,...,...,...,...,...,...
1340,2020-05-21,VITROX,1,1.418559,1.449847,1.438214,-0.257703,1.449488
1341,2020-09-11,VITROX,1,2.533699,2.438409,2.432330,0.540154,2.500931
1342,2020-11-27,VITROX,1,3.224771,3.202541,3.181903,-0.426413,3.262321
1343,2021-02-24,VITROX,0,3.826842,3.827740,3.846419,-0.301918,3.780274


In [11]:
df_all = result_scaler.sort_values(by="Buy_Date")
# setting Dates as index
df_all.set_index('Buy_Date', inplace=True)

In [12]:
# Encoding the stock names
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_all["Stock"] = le.fit_transform(df_all["Stock"])
df_all.head(20)

,Stock,wr,Buy,Low,Open,Volume,High
Buy_Date,,,,,,,
1994-09-07,7,0,-0.752361,-0.758173,-0.755265,1.202233,-0.733711
1994-09-27,7,0,-0.733371,-0.746831,-0.746833,0.809197,-0.714979
1994-09-30,7,0,-0.722520,-0.755337,-0.729968,-0.396468,-0.723007
1994-09-30,0,0,0.649960,0.685527,0.672256,-0.329433,0.656679
1994-10-04,0,0,0.658738,0.676506,0.663335,-0.470362,0.635251
1994-10-06,7,0,-0.727946,-0.746831,-0.741211,-0.391156,-0.728359
1994-10-11,3,1,-0.496911,-0.568786,-0.577692,-0.091368,-0.507702
1994-10-11,7,0,-0.733371,-0.752502,-0.746833,-0.704523,-0.733711
1994-11-09,3,1,-0.483444,-0.457038,-0.467226,-0.572828,-0.494393


In [13]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [14]:
# Separate input features and target
#X = df_all.drop(['wr','Pct_change','Sell_Date','Sell','Volume','Low','high'], axis=1)
X = df_all.drop(['wr'], axis=1)
y = df_all.wr

In [15]:
df_all["wr"].value_counts()

1    846
0    499
Name: wr, dtype: int64

In [16]:
# setting up testing and training sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=0, shuffle=False)

In [19]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


start_y = 17  
end_y = 21

test_prec = []
train_acc = []
test_acc = []
for i in range((end_y + 1) - start_y):  # years remaining
    for j in range(1,12,1):  # iteratting from 1 - 12
        
        # Fill the string with zeros until it is 2 characters long  
        date1 = "20{}-{}-01".format(start_y + i, str(j).zfill(2))  # starting of the test set
        date2 = "20{}-{}-01".format(start_y + i, str(j + 2).zfill(2))  # contain in 1 month only
        
        if j == 11:
            date2 = "20{}-{}-01".format(start_y + (i + 2), str((j + 3) - j).zfill(2))  # contain in 1 month only
        
        if date2 < str(df_all.index[-1]):  # limiting the splitting for duration below the last date
            
            print("Start:", date1)
            print("End:", date2)

            # splitting train test
            X_train, y_train = X[:date1], y[:date1]
            X_test, y_test = X[date1:date2], y[date1:date2]
            print("#### {}, {} ####".format(len(X_train), len(X_test)))
            
            

            # defining the model
            model = XGBClassifier(verbosity=0)
            # training the model
            model.fit(X_train, y_train)
            
            # predicting the train and test set
            # train
            y_pred = model.predict(X_train)
            pred1 = [round(value) for value in y_pred]
            train_accuracy = accuracy_score(y_train, pred1)
            train_acc.append(train_accuracy)
            
            # test
            y_pred = model.predict(X_test)
            pred2 = [round(value) for value in y_pred]
            test_accuracy = accuracy_score(y_test, pred2)
            test_acc.append(test_accuracy)
            print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
            
            precision = precision_score(y_test, pred2)
            test_prec.append(precision)
            #prec["{}".format(date1)] =  test_prec
            print("Test Precision: %.2f%%" % (precision * 100.0))
            
            if i == 0:
                print("----------------------------- DONE ROUND {} -----------------------------".format(i*10 + j))
            else:
                print("----------------------------- DONE ROUND {} ----------------------------- \n".format(i*10 + j+(2*i)))

avg_1 = sum(train_acc)/len(train_acc)  # calculating the average accuracy of the train set
avg_2 = sum(test_acc)/len(test_acc)  # calculating the average accuracy of the test set
print("Overall Train accuracy is: {}".format(avg_1))
print("Overall Test accuracy is: {}".format(avg_2)) 

avg_3 = sum(test_prec)/len(test_prec)  # calculating the average accuracy of the train set
print("Overall Test precision is: {}".format(avg_3))

            
            
            
            
            
            

Start: 2017-01-01
End: 2017-03-01
#### 1040, 7 ####
Test Accuracy: 42.86%
Test Precision: 100.00%
----------------------------- DONE ROUND 1 -----------------------------
Start: 2017-02-01
End: 2017-04-01
#### 1044, 8 ####
Test Accuracy: 75.00%
Test Precision: 100.00%
----------------------------- DONE ROUND 2 -----------------------------
Start: 2017-03-01
End: 2017-05-01
#### 1047, 14 ####
Test Accuracy: 85.71%
Test Precision: 91.67%
----------------------------- DONE ROUND 3 -----------------------------
Start: 2017-04-01
End: 2017-06-01
#### 1052, 21 ####
Test Accuracy: 66.67%
Test Precision: 86.67%
----------------------------- DONE ROUND 4 -----------------------------
Start: 2017-05-01
End: 2017-07-01
#### 1060, 18 ####
Test Accuracy: 66.67%
Test Precision: 92.31%
----------------------------- DONE ROUND 5 -----------------------------
Start: 2017-06-01
End: 2017-08-01
#### 1073, 18 ####
Test Accuracy: 83.33%
Test Precision: 100.00%
----------------------------- DONE ROUND 6 ---